In [18]:
import pandas as pd
import numpy as np
import itertools
import requests
#!conda install -c conda-forge folium=0.5.0 --yes # if folium is not yet install uncomment
import folium # map rendering library
import geopy
import geocoder
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist

In [19]:
df_raw = pd.read_html('https://de.wikipedia.org/wiki/Portal:Rems-Murr-Kreis/St%C3%A4dte_und_Gemeinden')[0]
df_raw_comm = pd.read_html('https://de.wikipedia.org/wiki/Portal:Rems-Murr-Kreis/St%C3%A4dte_und_Gemeinden')[1]

In [20]:
#Cities
df_raw1 = df_raw[[1]].rename({1: 'City'}, axis=1)
df_raw2 = df_raw[[3]].rename({3: 'City'}, axis=1)
df_raw3 = df_raw1.append(df_raw2).reset_index(drop = True)
print(df_raw3)
#Communities
df_raw_comm1 = df_raw_comm[[1]].rename({1: 'City'}, axis=1)
df_raw_comm2 = df_raw_comm[[3]].rename({3: 'City'}, axis=1)
df_raw_comm2 = df_raw_comm[[5]].rename({5: 'City'}, axis=1)
df_raw_comm3 = df_raw_comm1.append(df_raw_comm2).reset_index(drop = True).dropna()
print(df_raw_comm3)

         City
0    Backnang
1    Fellbach
2   Murrhardt
3  Schorndorf
4  Waiblingen
5   Weinstadt
6    Welzheim
7   Winnenden
               City
0           Alfdorf
1       Allmersbach
2          Althütte
3            Aspach
4          Auenwald
5           Berglen
6       Burgstetten
7        Großerlach
8        Rudersberg
9       Schwaikheim
10      Spiegelberg
11         Sulzbach
12           Urbach
13  Weissach im Tal
14       Winterbach


In [21]:
def get_latlng(city):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}'.format(city))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [22]:
coordinates = [ get_latlng(City) for City in df_raw3["City"].tolist() ]
coordinates_comm = [ get_latlng(City) for City in df_raw_comm3["City"].tolist() ]
coordinates

[[48.948390000000074, 9.432710000000043],
 [48.80919000000006, 9.276310000000024],
 [48.979670000000056, 9.579630000000066],
 [48.80368000000004, 9.528910000000053],
 [48.83301000000006, 9.31355000000002],
 [48.81248000000005, 9.365080000000034],
 [48.876810000000035, 9.634480000000053],
 [48.87629000000004, 9.395530000000065]]

In [24]:
df_coordinates = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])
df_raw3['Latitude'] = df_coordinates['Latitude']
df_raw3['Longitude'] = df_coordinates['Longitude']

df_coordinates_comm = pd.DataFrame(coordinates_comm, columns=['Latitude', 'Longitude'])
df_raw_comm3['Latitude'] = df_coordinates_comm['Latitude']
df_raw_comm3['Longitude'] = df_coordinates_comm['Longitude']

In [25]:
df = df_raw3.copy()
df_label = list(itertools.repeat(1, len(df)))
df.insert(1, "TrueLabel", df_label, True) 

df_comm = df_raw_comm3.copy()
comm_label = list(itertools.repeat(0, len(df_comm)))
df_comm.insert(1, "TrueLabel", comm_label, True) 

print(df_comm)
print(df)

               City  TrueLabel  Latitude  Longitude
0           Alfdorf          0  48.84361    9.71911
1       Allmersbach          0  48.46127   12.44550
2          Althütte          0  48.91742    9.56998
3            Aspach          0  48.96682    9.39942
4          Auenwald          0  48.93619    9.49894
5           Berglen          0  48.86118    9.46932
6       Burgstetten          0  48.92967    9.37020
7        Großerlach          0  49.05239    9.51331
8        Rudersberg          0  48.88578    9.52762
9       Schwaikheim          0  48.87754    9.34914
10      Spiegelberg          0  49.03772    9.44633
11         Sulzbach          0  49.29882    7.05696
12           Urbach          0  48.80931    9.57940
13  Weissach im Tal          0  48.92973    9.47857
14       Winterbach          0  48.79982    9.47928
         City  TrueLabel  Latitude  Longitude
0    Backnang          1  48.94839    9.43271
1    Fellbach          1  48.80919    9.27631
2   Murrhardt          1  48.9

In [26]:
df_comm.at[1, 'Latitude'] = 48.9074083
df_comm.at[1, 'Longitude'] = 9.4677844
df_comm.at[11, 'Latitude'] = 49.002349
df_comm.at[11, 'Longitude'] = 9.5014253
print(df_comm)

               City  TrueLabel   Latitude  Longitude
0           Alfdorf          0  48.843610   9.719110
1       Allmersbach          0  48.907408   9.467784
2          Althütte          0  48.917420   9.569980
3            Aspach          0  48.966820   9.399420
4          Auenwald          0  48.936190   9.498940
5           Berglen          0  48.861180   9.469320
6       Burgstetten          0  48.929670   9.370200
7        Großerlach          0  49.052390   9.513310
8        Rudersberg          0  48.885780   9.527620
9       Schwaikheim          0  48.877540   9.349140
10      Spiegelberg          0  49.037720   9.446330
11         Sulzbach          0  49.002349   9.501425
12           Urbach          0  48.809310   9.579400
13  Weissach im Tal          0  48.929730   9.478570
14       Winterbach          0  48.799820   9.479280


In [27]:
map_rmk = folium.Map(location=[48.9074083, 9.4677844], zoom_start=10)

# add markers to map
for lat, lng, city in zip(df['Latitude'], df['Longitude'], df['City']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_rmk)  
    
# add markers to map
for lat, lng, city in zip(df_comm['Latitude'], df_comm['Longitude'], df_comm['City']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_rmk)      
    
map_rmk

In [28]:
frames = [df, df_comm]
df_rmk = pd.concat(frames)
print(df_rmk)

               City  TrueLabel   Latitude  Longitude
0          Backnang          1  48.948390   9.432710
1          Fellbach          1  48.809190   9.276310
2         Murrhardt          1  48.979670   9.579630
3        Schorndorf          1  48.803680   9.528910
4        Waiblingen          1  48.833010   9.313550
5         Weinstadt          1  48.812480   9.365080
6          Welzheim          1  48.876810   9.634480
7         Winnenden          1  48.876290   9.395530
0           Alfdorf          0  48.843610   9.719110
1       Allmersbach          0  48.907408   9.467784
2          Althütte          0  48.917420   9.569980
3            Aspach          0  48.966820   9.399420
4          Auenwald          0  48.936190   9.498940
5           Berglen          0  48.861180   9.469320
6       Burgstetten          0  48.929670   9.370200
7        Großerlach          0  49.052390   9.513310
8        Rudersberg          0  48.885780   9.527620
9       Schwaikheim          0  48.877540   9.

In [31]:
#CLIENT_ID = '***YOUR CREDENTIALS***'
#CLIENT_SECRET = '***YOUR CREDENTIALS***'
#VERSION = '20200731'

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [48]:
radius = 1500
limit = 50

venues = []

for lat, long, city in zip(df_rmk['Latitude'], 
                           df_rmk['Longitude'],
                           df_rmk['City']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        limit)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            city,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [49]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['City', 'CityLatitude', 'CityLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
print(venues_df)

(218, 7)
                City  CityLatitude  CityLongitude  \
0           Backnang      48.94839        9.43271   
1           Backnang      48.94839        9.43271   
2           Backnang      48.94839        9.43271   
3           Backnang      48.94839        9.43271   
4           Backnang      48.94839        9.43271   
..               ...           ...            ...   
213  Weissach im Tal      48.92973        9.47857   
214       Winterbach      48.79982        9.47928   
215       Winterbach      48.79982        9.47928   
216       Winterbach      48.79982        9.47928   
217       Winterbach      48.79982        9.47928   

                                  VenueName  VenueLatitude  VenueLongitude  \
0                                     McFIT      48.947344        9.433183   
1                                      Lidl      48.946760        9.436951   
2                         dm-drogerie markt      48.948013        9.429857   
3             Schürers Restaurant Tafelhau

In [50]:
print(venues_df.groupby(["City"]).count())

                 CityLatitude  CityLongitude  VenueName  VenueLatitude  \
City                                                                     
Alfdorf                     4              4          4              4   
Allmersbach                 4              4          4              4   
Althütte                    6              6          6              6   
Aspach                      3              3          3              3   
Auenwald                    6              6          6              6   
Backnang                   19             19         19             19   
Berglen                     5              5          5              5   
Burgstetten                 4              4          4              4   
Fellbach                   43             43         43             43   
Großerlach                  4              4          4              4   
Murrhardt                   5              5          5              5   
Rudersberg                  6         

In [51]:
# one hot encoding
rmk_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
rmk_onehot['City'] = venues_df['City'] 
rmk_grouped = rmk_onehot.groupby(["City"]).mean().reset_index()

print(rmk_grouped.shape)
rmk_grouped

(23, 80)


,City,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,Bakery,Bank,Beach Bar,Beer Garden,Big Box Store,...,Soccer Field,Soccer Stadium,Steakhouse,Supermarket,Theme Park Ride / Attraction,Train Station,Video Store,Water Park,Wine Shop,Women's Store
0,Alfdorf,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,Allmersbach,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,Althütte,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Aspach,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Auenwald,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,Backnang,0.000000,0.000000,0.052632,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.0,0.000000,0.000000,0.105263,0.000000,0.000000,0.052632,0.000000,0.000000,0.00
6,Berglen,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.0,0.000000,0.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Burgstetten,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,Fellbach,0.023256,0.023256,0.000000,0.000000,0.023256,0.00,0.00,0.000000,0.023256,...,0.0,0.023256,0.000000,0.046512,0.023256,0.000000,0.000000,0.000000,0.023256,0.00
9,Großerlach,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.00,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [52]:
# set number of clusters
kclusters = 2

rmk_grouped_clustering = rmk_grouped.drop(['City'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(rmk_grouped_clustering )

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [55]:
#df_rmk.drop(['ClusterLabels'], axis=1)
df_rmk.insert(0, 'Cluster_Labels', kmeans.labels_)
df_rmk

# create a new dataframe that includes the cluster as well as the top 20 venues for each neighborhood.
#toronto_merged = toronto_central_df.copy()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.drop(["Borough", "Neighbourhoods"], 1).set_index("PostalCode"), on="PostalCode")

#print(toronto_merged.shape)
#toronto_merged.head()

,Cluster_Labels,ClusterLabels,City,TrueLabel,Latitude,Longitude,Change
0,0,0,Backnang,1,48.948390,9.432710,2
1,1,0,Fellbach,1,48.809190,9.276310,2
2,0,0,Murrhardt,1,48.979670,9.579630,2
3,1,1,Schorndorf,1,48.803680,9.528910,1
4,1,0,Waiblingen,1,48.833010,9.313550,2
5,0,0,Weinstadt,1,48.812480,9.365080,2
6,0,0,Welzheim,1,48.876810,9.634480,2
7,1,0,Winnenden,1,48.876290,9.395530,2
0,0,0,Alfdorf,0,48.843610,9.719110,0
1,0,0,Allmersbach,0,48.907408,9.467784,0


In [78]:
df_rmk['Change'] = np.where((df_rmk['TrueLabel'] == df_rmk['Cluster_Labels']), df_rmk['TrueLabel'], 2)
df_rmk

,Cluster_Labels,ClusterLabels,City,TrueLabel,Latitude,Longitude,Change
0,0,0,Backnang,1,48.948390,9.432710,2
1,1,0,Fellbach,1,48.809190,9.276310,1
2,0,0,Murrhardt,1,48.979670,9.579630,2
3,1,1,Schorndorf,1,48.803680,9.528910,1
4,1,0,Waiblingen,1,48.833010,9.313550,1
5,0,0,Weinstadt,1,48.812480,9.365080,2
6,0,0,Welzheim,1,48.876810,9.634480,2
7,1,0,Winnenden,1,48.876290,9.395530,1
0,0,0,Alfdorf,0,48.843610,9.719110,0
1,0,0,Allmersbach,0,48.907408,9.467784,0


In [79]:
df_rmk.loc[df_rmk.City == 'Backnang', 'Change'] = 3
df_rmk.loc[df_rmk.City == 'Murrhardt', 'Change'] = 3
df_rmk.loc[df_rmk.City == 'Weinstadt', 'Change'] = 3
df_rmk.loc[df_rmk.City == 'Welzheim', 'Change'] = 3
df_rmk

,Cluster_Labels,ClusterLabels,City,TrueLabel,Latitude,Longitude,Change
0,0,0,Backnang,1,48.948390,9.432710,3
1,1,0,Fellbach,1,48.809190,9.276310,1
2,0,0,Murrhardt,1,48.979670,9.579630,3
3,1,1,Schorndorf,1,48.803680,9.528910,1
4,1,0,Waiblingen,1,48.833010,9.313550,1
5,0,0,Weinstadt,1,48.812480,9.365080,3
6,0,0,Welzheim,1,48.876810,9.634480,3
7,1,0,Winnenden,1,48.876290,9.395530,1
0,0,0,Alfdorf,0,48.843610,9.719110,0
1,0,0,Allmersbach,0,48.907408,9.467784,0


In [80]:
rmk_cluster1 = df_rmk[df_rmk['Change'] == 0]
rmk_cluster2 = df_rmk[df_rmk['Change'] == 1]
rmk_cluster3 = df_rmk[df_rmk['Change'] == 2]
rmk_cluster4 = df_rmk[df_rmk['Change'] == 3]

In [82]:
map_rmk = folium.Map(location=[48.9074083, 9.4677844], zoom_start=10)

# add markers to map
for lat, lng, city in zip(rmk_cluster1['Latitude'], rmk_cluster1['Longitude'], rmk_cluster1['City']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_rmk)  
    
# add markers to map
for lat, lng, city in zip(rmk_cluster2['Latitude'], rmk_cluster2['Longitude'], rmk_cluster2['City']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_rmk)
    
# add markers to map
for lat, lng, city in zip(rmk_cluster3['Latitude'], rmk_cluster3['Longitude'], rmk_cluster3['City']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='red',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_rmk)

# add markers to map
for lat, lng, city in zip(rmk_cluster4['Latitude'], rmk_cluster4['Longitude'], rmk_cluster4['City']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_rmk)       
    
map_rmk